In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency, metrics

import helper

np.random.seed(247)
tf.set_random_seed(247)

# Load dataset

In [ ]:
# dataset path# save path
results_path = utils.make_directory('../results', 'synthetic')
params_path = utils.make_directory(results_path, 'model_params')

# load dataset
data_path = '../data/synthetic_dataset.h5'
train, valid, test = helper.load_synthetic_dataset(data_path)

test_model = helper.load_synthetic_TF_models(data_path, dataset='test')
    
    
# get data shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = [None, train['targets'].shape[1]]

# Load model

In [ ]:
tf.reset_default_graph()

model_name = 'cnn_2'
# model_name = 'cnn_25_4'

# load model parameters
genome_model = helper.import_model(model_name)
model_layers, optimization = genome_model.model(input_shape, output_shape)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, supervised=True)
nnmodel.inspect_layers()

# create neural trainer
file_path = os.path.join(params_path, model_name)
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session()

# load best parameters
nntrainer.set_best_parameters(sess)

# get the model predictions (activations) and the logits (threshold)
activations = nntrainer.get_activations(sess, test, layer='output')
threshold = nntrainer.get_activations(sess, test, layer=nnmodel.network.keys()[-2])

sess.close()

# Plot top saliency maps for each class

In [ ]:
save_path = utils.make_directory(results_path, 'saliency')

# get number of class labels for each sequence
class_sum = np.sum(test['targets'], axis=1)

num_plots = 5

# loop through different transcription factors
for class_index in range(12)[::-1]:
    
    # get indices of a sequence that has a positive label and has a single class label
    right_indices = np.where((test['targets'][:,class_index]==1) & 
                                   (class_sum == 1)
                                  )[0]

    # sort by most confident predictions
    right_indices = right_indices[np.argsort(activations[right_indices, class_index])[::-1]][:num_plots]

    for right_index in right_indices:
        
        # get the sequence and the sequence model
        X = np.expand_dims(test['inputs'][right_index], axis=0)
        X_model = test_model[right_index]
        
        # get saliency map for CNN-2
        model_name = 'cnn_2'
        genome_model = helper.import_model(model_name)
        file_path = os.path.join(params_path, model_name)
        params = {'model_name': model_name, 
                  'genome_model': genome_model.model,
                  'input_shape': input_shape, 
                  'output_shape': output_shape, 
                  'model_path': file_path+'_best.ckpt',
                 }
        guided_saliency = saliency.guided_backprop(X, layer='output', class_index=class_index, params=params)


        # get saliency map for CNN3_2
        model_name = 'cnn3_2'
        genome_model = helper.import_model(model_name)
        file_path = os.path.join(params_path, model_name)
        params = {'model_name': model_name, 
                  'genome_model': genome_model.model,
                  'input_shape': input_shape, 
                  'output_shape': output_shape, 
                  'model_path': file_path+'_best.ckpt',
                 }
        guided_saliency2 = saliency.guided_backprop(X, layer='output', class_index=class_index, params=params)

        # get saliency map for CNN-25
        model_name = 'cnn_25'
        genome_model = helper.import_model(model_name)
        file_path = os.path.join(params_path, model_name)
        params = {'model_name': model_name, 
                  'genome_model': genome_model.model,
                  'input_shape': input_shape, 
                  'output_shape': output_shape, 
                  'model_path': file_path+'_best.ckpt',
                 }
        shallow_guided_saliency = saliency.guided_backprop(X, layer='output', class_index=class_index, params=params)

        # sequence logo comparison between different CNN models
        fig = plt.figure(figsize=(50,7))
        plt.subplot(3,1,1)
        visualize.plot_pos_saliency(np.squeeze(shallow_guided_saliency).T, alphabet='dna',height=150, nt_width=50, norm_factor=3)
        plt.axis('on')
        plt.xticks([])
        plt.yticks([])
        plt.ylabel('CNN-25', fontsize=36)
        plt.subplot(3,1,2)
        visualize.plot_pos_saliency(np.squeeze(guided_saliency).T, alphabet='dna',height=150, nt_width=50, norm_factor=3)
        plt.axis('on')
        plt.xticks([])
        plt.yticks([])
        plt.ylabel('CNN-2', fontsize=36)
        plt.subplot(3,1,3)
        visualize.plot_pos_saliency(np.squeeze(guided_saliency2).T, alphabet='dna',height=150, nt_width=50, norm_factor=3)
        plt.axis('on')
        plt.xticks([])
        plt.yticks([])
        plt.ylabel('CNN3-2', fontsize=36)
        outfile = os.path.join(save_path, str(class_index)+'_comparison_'+str(right_index)+'.pdf')
        fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight') 

        # save the sequence model and the sequence
        fig = plt.figure(figsize=(50,3))
        visualize.plot_seq_pos_saliency(np.squeeze(X).T, X_model, alphabet='dna', nt_width=50, norm_factor=3)
        outfile = os.path.join(save_path, str(class_index)+'_model_'+str(right_index)+'.pdf')
        fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight') 
